In [6]:
import pandas as pd
import numpy as np
from numpy._core.defchararray import count
import matplotlib.pyplot as plt
from scipy import stats
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
import random

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
random.seed(RANDOM_STATE)

df_pesticides = pd.read_csv('/content/drive/MyDrive/AI4ALL/Datasets | XG Boost Model/historical_data_1974_2022_enriched.csv')
df_confounders = pd.read_csv('/content/drive/MyDrive/AI4ALL/Datasets | XG Boost Model/copd_aqi_poverty_demographics.csv')

df_pesticides2 = df_pesticides.drop(columns=['SOURCE_FILE', 'APPLICATION_METHOD_DESC', 'APPLICATION_METHOD', 'TYPE_CODE', 'USE_CODE', 'FORM_CODE', 'RECORD_ID', 'COUNTY_CD', 'SITE_CODE','CHEM_NAME','SITE_NAME'])

print(f'Pesticides: {df_pesticides2.columns.tolist()}')
print(f'Confounders: {df_confounders.columns.tolist()}')

print(f'\nPesticide shape: {df_pesticides2.shape}')
print(f'Confounders shape: {df_confounders.shape}')

print(f'\nPesticide yrs: {df_pesticides2['YEAR'].min()}--{df_pesticides2['YEAR'].max()} ')
print(f'Confounders yrs: {df_confounders['Year'].min()}--{df_confounders['Year'].max()}')

/tmp/ipython-input-2376248988.py:15: DtypeWarning: Columns (4,5,6,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df_pesticides = pd.read_csv('/content/drive/MyDrive/AI4ALL/Datasets | XG Boost Model/historical_data_1974_2022_enriched.csv')


Pesticides: ['YEAR', 'CHEM_CODE', 'TOTAL_LBS_AI', 'TOTAL_ACRES_TREATED', 'COUNTY_NAME']
Confounders: ['Counties', 'Year', 'Median AQI', 'pct_under_18', 'pct_18_64', 'pct_65_plus', 'median_age', 'pct_AI/AN', 'pct_Asian', 'pct_Black', 'pct_Latino', 'pct_Multi_Race', 'pct_NH/PI', 'pct_White', 'COPD_Hospitalization_Rate', 'Poverty_AllAges_Percent_Est', 'Median_Household_Income_Est']

Pesticide shape: (2001180, 5)
Confounders shape: (1272, 17)

Pesticide yrs: 1974--2022 
Confounders yrs: 2000--2023


In [7]:
#Data Standardization

df_pesticides2.columns = map(str.lower, df_pesticides2.columns)
df_confounders.columns = map(str.lower, df_confounders.columns)
df_pesticides2 = df_pesticides2.rename(columns={'county_name': 'county', 'total_lbs_ai': 'total_pesticide_lbs'})
df_confounders = df_confounders.rename(columns={'counties': 'county'})
df_pesticides2['county'] = df_pesticides2['county'].replace('Tuolomne','Tuolumne')
df_confounders['county'] = df_confounders['county'].replace('Tuolomne','Tuolumne')
df_pesticides2['county'] = df_pesticides2['county'].str.lower()
df_confounders['county'] = df_confounders['county'].str.lower()
df_pesticides2['county'] = df_pesticides2['county'].str.strip()
df_confounders['county'] = df_confounders['county'].str.strip()




df_confounders['median aqi'] = df_confounders.groupby('county')['median aqi'].transform(
    lambda x: x.fillna(x.mean())
)
df_confounders['median aqi'] = df_confounders['median aqi'].fillna(
    df_confounders['median aqi'].mean()
)



pesticide_counties = set(df_pesticides2['county'].unique())
confounding_counties = set(df_confounders['county'].unique())
county_assessment = pesticide_counties - confounding_counties

print(f'\nCounties not in one or the other: {county_assessment}')
print(f'\nCounties to be excluded: {county_assessment}')
print(f'Overlapping counties: {len(pesticide_counties & confounding_counties)}')
print(f'\nPesticides: {df_pesticides2.columns.tolist()}')
print(f'Confounders: {df_confounders.columns.tolist()}')
print(f'\nNulls for pesticide data: {df_pesticides2.isnull().sum()}')
print(f'\nNulls for confounders data: {df_confounders.isnull().sum()}')



Counties not in one or the other: {'modoc', 'alpine', 'sierra', 'mono', 'lassen'}

Counties to be excluded: {'modoc', 'alpine', 'sierra', 'mono', 'lassen'}
Overlapping counties: 53

Pesticides: ['year', 'chem_code', 'total_pesticide_lbs', 'total_acres_treated', 'county']
Confounders: ['county', 'year', 'median aqi', 'pct_under_18', 'pct_18_64', 'pct_65_plus', 'median_age', 'pct_ai/an', 'pct_asian', 'pct_black', 'pct_latino', 'pct_multi_race', 'pct_nh/pi', 'pct_white', 'copd_hospitalization_rate', 'poverty_allages_percent_est', 'median_household_income_est']

Nulls for pesticide data: year                   0
chem_code              0
total_pesticide_lbs    0
total_acres_treated    0
county                 0
dtype: int64

Nulls for confounders data: county                         0
year                           0
median aqi                     0
pct_under_18                   0
pct_18_64                      0
pct_65_plus                    0
median_age                     0
pct_ai/an 

In [8]:
#Population Data

df_population = pd.read_csv('/content/drive/MyDrive/AI4ALL/Datasets | XG Boost Model/Population_Census_Numbers_2000_2025.csv')

df_population = df_population.rename(columns={'County': 'county'})

population_long = df_population.melt(
    id_vars=['county'],
    var_name='date_str',
    value_name='total_population'
)

def year_rewrite(date_str):
  year_num = int(date_str.split('/')[-1])

  if year_num >= 1900:
    return year_num
  elif year_num <= 25:
    return 2000 + year_num
  else:
    return 1900 + year_num

population_long['year'] = population_long['date_str'].apply(year_rewrite)

population_long2 = population_long.drop(columns=['date_str'])

population_long2['total_population'] = population_long2['total_population'].str.replace('\t','')
population_long2['total_population'] = population_long2['total_population'].str.replace(',','')
population_long2['total_population'] = population_long2['total_population'].astype(float)
population_long2['county'] = population_long2['county'].str.lower()
population_long2['county'] = population_long2['county'].str.strip()


print(f'\nPopulation shape: {population_long2.shape}')
print('\nCounties:', population_long2['county'].nunique())
population_long2.head(11)


Population shape: (1508, 3)

Counties: 58


,county,total_population,year
0,alameda,1437136.0,2000
1,alpine,1196.0,2000
2,amador,34990.0,2000
3,butte,202658.0,2000
4,calaveras,40465.0,2000
5,colusa,18734.0,2000
6,contra costa,944953.0,2000
7,del norte,27509.0,2000
8,el dorado,155702.0,2000
9,fresno,796187.0,2000


In [9]:
# Aggregating Pesticide Data by County-Year

rows_by_county_year = df_pesticides2.groupby(['year','county']).size().head(6)
print(rows_by_county_year)

df_pesticides2 = df_pesticides2[(df_pesticides2['year'] >= 2000) & (df_pesticides2['year'] <= 2022)]
df_confounders = df_confounders[(df_confounders['year'] >= 2000) & (df_confounders['year'] <= 2022)]


pesticide_overall = df_pesticides2.groupby(['county', 'year']).agg({
    'total_pesticide_lbs': 'sum',
    'total_acres_treated': 'sum'
}).reset_index()



print(f'Total observations: {len(pesticide_overall)}')
print(f'Shape: {pesticide_overall.shape}')
print(f'Year range: {pesticide_overall['year'].min()} - {pesticide_overall['year'].max()}')
print(f'Counties: {pesticide_overall['county'].nunique()}')
print('\nFirst 10 rows:')
print(pesticide_overall.head(10))


year  county   
1974  alameda      308
      alpine         4
      amador        26
      butte        595
      calaveras     41
      colusa       496
dtype: int64
Total observations: 1316
Shape: (1316, 4)
Year range: 2000 - 2022
Counties: 58

First 10 rows:
    county  year  total_pesticide_lbs  total_acres_treated
0  alameda  2000        127328.522952        286372.873803
1  alameda  2001         50457.765421        266309.298930
2  alameda  2002         67414.358828         97443.188425
3  alameda  2003         30453.333603         34543.221818
4  alameda  2004         38935.451938         35817.187925
5  alameda  2005         51177.144141         55303.923159
6  alameda  2006         53303.939136         51863.412773
7  alameda  2007         68056.880288         48208.728460
8  alameda  2008        124013.315103         51974.196267
9  alameda  2009         16777.030140         23529.570000


In [10]:
#Normalizing Pesticide data by population

population_long2 = population_long2[(population_long2['year'] >= 2000) & (population_long2['year'] <= 2022)]

pesticide_overall_merged = pesticide_overall.merge(population_long2,on=['county','year'],how='left')

pesticide_overall_merged['total_pesticide_lbs_per_100k'] = ((pesticide_overall_merged['total_pesticide_lbs']/pesticide_overall_merged['total_population'])*100000)
pesticide_overall_merged['total_acres_treated_per_100k'] = ((pesticide_overall_merged['total_acres_treated']/pesticide_overall_merged['total_population'])*100000)


pesticide_overall_merged_clean = pesticide_overall_merged.dropna(subset=['total_population'])
print(f'Rows dropped: {len(pesticide_overall_merged) - len(pesticide_overall_merged_clean)}')

print('\nFinal summary statistics:')
print(pesticide_overall_merged_clean[['total_pesticide_lbs_per_100k', 'total_acres_treated_per_100k']].describe())
print(pesticide_overall_merged_clean.head(10))
pesticide_overall_merged = pesticide_overall_merged_clean


Rows dropped: 0

Final summary statistics:
       total_pesticide_lbs_per_100k  total_acres_treated_per_100k
count                  1.316000e+03                  1.316000e+03
mean                   1.181447e+06                  6.110174e+05
std                    1.840960e+06                  1.201151e+06
min                    1.923104e+01                  2.515850e+00
25%                    2.665776e+04                  1.927044e+04
50%                    4.507967e+05                  1.285501e+05
75%                    1.438603e+06                  6.348758e+05
max                    1.247004e+07                  9.414095e+06
    county  year  total_pesticide_lbs  total_acres_treated  total_population  \
0  alameda  2000        127328.522952        286372.873803         1437136.0   
1  alameda  2001         50457.765421        266309.298930         1457185.0   
2  alameda  2002         67414.358828         97443.188425         1467063.0   
3  alameda  2003         30453.333603      

In [11]:
#Creating Lag Features

def lag_features(df, column, lag_years=[1, 2, 3, 5,10,15,20]):
  df_merged_sorted = pesticide_overall_merged.sort_values(['county','year']).copy()

  for col in column:
    for lag in lag_years:
      df_merged_sorted[f'{col}_lag{lag}'] = df_merged_sorted.groupby('county')[col].shift(lag)
  return df_merged_sorted

to_lag = ['total_pesticide_lbs_per_100k','total_acres_treated_per_100k']

pesticides_with_lags = lag_features(pesticide_overall_merged, to_lag, lag_years=[1, 2, 3, 5, 10, 15, 20])


print(f'Original columns: {len(pesticide_overall_merged.columns)}')
print(f'Columns with lags: {len(pesticides_with_lags.columns)}')
print(f'New lag columns added: {len(pesticides_with_lags.columns) - len(pesticide_overall_merged.columns)}')

print(f'All columns: {pesticides_with_lags.columns.to_list()}')

pesticides_with_lags.head(6)


Original columns: 7
Columns with lags: 21
New lag columns added: 14
All columns: ['county', 'year', 'total_pesticide_lbs', 'total_acres_treated', 'total_population', 'total_pesticide_lbs_per_100k', 'total_acres_treated_per_100k', 'total_pesticide_lbs_per_100k_lag1', 'total_pesticide_lbs_per_100k_lag2', 'total_pesticide_lbs_per_100k_lag3', 'total_pesticide_lbs_per_100k_lag5', 'total_pesticide_lbs_per_100k_lag10', 'total_pesticide_lbs_per_100k_lag15', 'total_pesticide_lbs_per_100k_lag20', 'total_acres_treated_per_100k_lag1', 'total_acres_treated_per_100k_lag2', 'total_acres_treated_per_100k_lag3', 'total_acres_treated_per_100k_lag5', 'total_acres_treated_per_100k_lag10', 'total_acres_treated_per_100k_lag15', 'total_acres_treated_per_100k_lag20']


,county,year,total_pesticide_lbs,total_acres_treated,total_population,total_pesticide_lbs_per_100k,total_acres_treated_per_100k,total_pesticide_lbs_per_100k_lag1,total_pesticide_lbs_per_100k_lag2,total_pesticide_lbs_per_100k_lag3,...,total_pesticide_lbs_per_100k_lag10,total_pesticide_lbs_per_100k_lag15,total_pesticide_lbs_per_100k_lag20,total_acres_treated_per_100k_lag1,total_acres_treated_per_100k_lag2,total_acres_treated_per_100k_lag3,total_acres_treated_per_100k_lag5,total_acres_treated_per_100k_lag10,total_acres_treated_per_100k_lag15,total_acres_treated_per_100k_lag20
0,alameda,2000,127328.522952,286372.873803,1437136.0,8859.879855,19926.636992,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,alameda,2001,50457.765421,266309.298930,1457185.0,3462.687677,18275.599799,8859.879855,NaN,NaN,...,NaN,NaN,NaN,19926.636992,NaN,NaN,NaN,NaN,NaN,NaN
2,alameda,2002,67414.358828,97443.188425,1467063.0,4595.191810,6642.058891,3462.687677,8859.879855,NaN,...,NaN,NaN,NaN,18275.599799,19926.636992,NaN,NaN,NaN,NaN,NaN
3,alameda,2003,30453.333603,34543.221818,1467892.0,2074.630395,2353.253633,4595.191810,3462.687677,8859.879855,...,NaN,NaN,NaN,6642.058891,18275.599799,19926.636992,NaN,NaN,NaN,NaN
4,alameda,2004,38935.451938,35817.187925,1466407.0,2655.159989,2442.513431,2074.630395,4595.191810,3462.687677,...,NaN,NaN,NaN,2353.253633,6642.058891,18275.599799,NaN,NaN,NaN,NaN
5,alameda,2005,51177.144141,55303.923159,1462736.0,3498.727326,3780.854724,2655.159989,2074.630395,4595.191810,...,NaN,NaN,NaN,2442.513431,2353.253633,6642.058891,19926.636992,NaN,NaN,NaN


In [12]:
#Creating cumulative Exposure Features

def cumulative_features(df, column, windows=[3,5,10,15,20]):
  df_copied = df.copy()

  for col in column:
    for window in windows:
      df_copied[f'{col}_cumulative_sum{window}year'] = (df_copied.groupby('county')[col].rolling(window=window, min_periods=1).sum().reset_index(level=0, drop=True))
      df_copied[f'{col}_cumulative_mean{window}year'] = (df_copied.groupby('county')[col].rolling(window=window, min_periods=1).mean().reset_index(level=0, drop=True))
  return df_copied


to_accumulate = ['total_pesticide_lbs_per_100k','total_acres_treated_per_100k']

final_pesticide = cumulative_features(pesticides_with_lags, to_accumulate, windows=[3,5,10,15,20])


print(f'Columns before: {len(pesticides_with_lags.columns)}')
print(f'Columns after: {len(final_pesticide.columns)}')
print(f'New features added: {len(final_pesticide.columns) - len(pesticides_with_lags.columns)}')

final_pesticide.head(6)


Columns before: 21
Columns after: 41
New features added: 20


,county,year,total_pesticide_lbs,total_acres_treated,total_population,total_pesticide_lbs_per_100k,total_acres_treated_per_100k,total_pesticide_lbs_per_100k_lag1,total_pesticide_lbs_per_100k_lag2,total_pesticide_lbs_per_100k_lag3,...,total_acres_treated_per_100k_cumulative_sum3year,total_acres_treated_per_100k_cumulative_mean3year,total_acres_treated_per_100k_cumulative_sum5year,total_acres_treated_per_100k_cumulative_mean5year,total_acres_treated_per_100k_cumulative_sum10year,total_acres_treated_per_100k_cumulative_mean10year,total_acres_treated_per_100k_cumulative_sum15year,total_acres_treated_per_100k_cumulative_mean15year,total_acres_treated_per_100k_cumulative_sum20year,total_acres_treated_per_100k_cumulative_mean20year
0,alameda,2000,127328.522952,286372.873803,1437136.0,8859.879855,19926.636992,NaN,NaN,NaN,...,19926.636992,19926.636992,19926.636992,19926.636992,19926.636992,19926.636992,19926.636992,19926.636992,19926.636992,19926.636992
1,alameda,2001,50457.765421,266309.298930,1457185.0,3462.687677,18275.599799,8859.879855,NaN,NaN,...,38202.236791,19101.118396,38202.236791,19101.118396,38202.236791,19101.118396,38202.236791,19101.118396,38202.236791,19101.118396
2,alameda,2002,67414.358828,97443.188425,1467063.0,4595.191810,6642.058891,3462.687677,8859.879855,NaN,...,44844.295682,14948.098561,44844.295682,14948.098561,44844.295682,14948.098561,44844.295682,14948.098561,44844.295682,14948.098561
3,alameda,2003,30453.333603,34543.221818,1467892.0,2074.630395,2353.253633,4595.191810,3462.687677,8859.879855,...,27270.912323,9090.304108,47197.549315,11799.387329,47197.549315,11799.387329,47197.549315,11799.387329,47197.549315,11799.387329
4,alameda,2004,38935.451938,35817.187925,1466407.0,2655.159989,2442.513431,2074.630395,4595.191810,3462.687677,...,11437.825955,3812.608652,49640.062746,9928.012549,49640.062746,9928.012549,49640.062746,9928.012549,49640.062746,9928.012549
5,alameda,2005,51177.144141,55303.923159,1462736.0,3498.727326,3780.854724,2655.159989,2074.630395,4595.191810,...,8576.621788,2858.873929,33494.280478,6698.856096,53420.917470,8903.486245,53420.917470,8903.486245,53420.917470,8903.486245


In [13]:
# Merging Pesticide Features with Confounders

final_merged_with_confounders = df_confounders.merge(final_pesticide,on=['county','year'],how='inner')
complete_dataset = final_merged_with_confounders


county_coverage = complete_dataset.groupby('county')['year'].agg(['count', 'min', 'max'])
county_coverage.columns = ['num_years', 'first_year', 'last_year']
county_coverage = county_coverage.sort_values('num_years')

print(f'County Coverage Summary:{county_coverage}')
print(f'\nCounties with complete data (23 years): {(county_coverage['num_years'] == 23).sum()}')
print(f'Counties with partial data: {(county_coverage['num_years'] < 23).sum()}')






County Coverage Summary:                 num_years  first_year  last_year
county                                           
san francisco           14        2008       2022
inyo                    21        2001       2022
butte                   23        2000       2022
alameda                 23        2000       2022
colusa                  23        2000       2022
contra costa            23        2000       2022
del norte               23        2000       2022
amador                  23        2000       2022
el dorado               23        2000       2022
fresno                  23        2000       2022
humboldt                23        2000       2022
glenn                   23        2000       2022
imperial                23        2000       2022
kern                    23        2000       2022
kings                   23        2000       2022
calaveras               23        2000       2022
los angeles             23        2000       2022
madera                  23

In [14]:
# Handling missing values

complete_dataset_filtered = complete_dataset[complete_dataset['year'] >= 2005].copy()


lag10_cols = [col for col in complete_dataset_filtered.columns if 'lag10' in col]
lag15_cols = [col for col in complete_dataset_filtered.columns if 'lag15' in col]
lag20_cols = [col for col in complete_dataset_filtered.columns if 'lag20' in col]
complete_dataset_filtered = complete_dataset_filtered.drop(columns=lag20_cols)
complete_dataset_filtered = complete_dataset_filtered.drop(columns=lag15_cols)
complete_dataset_filtered = complete_dataset_filtered.drop(columns=lag10_cols)


complete_dataset_cleaned = complete_dataset_filtered.dropna()
print('Cleaned Dataset summarization')
print(f'- Shape: {complete_dataset_cleaned.shape}')
print(f'- Year range: {complete_dataset_cleaned['year'].min()} - {complete_dataset_cleaned['year'].max()}')
print(f'- Counties: {complete_dataset_cleaned['county'].nunique()}')
print(f'- Total features: {complete_dataset_cleaned.shape[1] - 3}')
print(f'- Observations per feature: {len(complete_dataset_cleaned) / (complete_dataset_cleaned.shape[1] - 3):.1f}')
print(complete_dataset_cleaned['copd_hospitalization_rate'].describe())




Cleaned Dataset summarization
  - Shape: (943, 50)
  - Year range: 2005 - 2022
  - Counties: 53
  - Total features: 47
  - Observations per feature: 20.1
count    943.000000
mean      16.330870
std        8.804357
min        1.620000
25%       10.255000
50%       15.080000
75%       20.380000
max       58.650000
Name: copd_hospitalization_rate, dtype: float64


In [15]:
#Validating Lag features

county_counts = final_pesticide.groupby('county').size()
sample_county = county_counts.idxmax()

validation_sample = final_pesticide[final_pesticide['county'] == sample_county].sort_values('year')

display_cols = ['year', 'total_pesticide_lbs_per_100k',
                'total_pesticide_lbs_per_100k_lag1',
                'total_pesticide_lbs_per_100k_lag5',
                'total_pesticide_lbs_per_100k_lag10',
                'total_pesticide_lbs_per_100k_lag20']

print(validation_sample[display_cols].head(25).to_string(index=False))

 year  total_pesticide_lbs_per_100k  total_pesticide_lbs_per_100k_lag1  total_pesticide_lbs_per_100k_lag5  total_pesticide_lbs_per_100k_lag10  total_pesticide_lbs_per_100k_lag20
 2000                   8859.879855                                NaN                                NaN                                 NaN                                 NaN
 2001                   3462.687677                        8859.879855                                NaN                                 NaN                                 NaN
 2002                   4595.191810                        3462.687677                                NaN                                 NaN                                 NaN
 2003                   2074.630395                        4595.191810                                NaN                                 NaN                                 NaN
 2004                   2655.159989                        2074.630395                                NaN     

In [16]:
# Feature Organization

indentifiers = ['county','years']
target = 'copd_hospitalization_rate'

confounding_features = [
    'median aqi',
    'pct_under_18', 'pct_18_64', 'pct_65_plus', 'median_age',
    'pct_ai/an', 'pct_asian', 'pct_black', 'pct_latino',
    'pct_multi_race', 'pct_nh/pi', 'pct_white',
    'poverty_allages_percent_est', 'median_household_income_est'
]
pesticide_features = ['total_pesticide_lbs', 'total_acres_treated',
                 'total_population', 'total_pesticide_lbs_per_100k',
                 'total_acres_treated_per_100k']

pesticide_lag = [col for col in complete_dataset_cleaned.columns if 'lag' in col]
pesticide_cumulative = [col for col in complete_dataset_cleaned.columns if 'cumulative' in col]


features_to_drop = []
sum_features = [col for col in complete_dataset_cleaned.columns if 'cumulative_sum' in col]
features_to_drop.extend(sum_features)
lag_to_drop = [col for col in complete_dataset_cleaned.columns if 'lag3' in col or 'lag5' in col]
features_to_drop.extend(lag_to_drop)
cum_mean_to_drop = [col for col in complete_dataset_cleaned.columns
                    if any(x in col for x in ['cumulative_mean3year',
                                               'cumulative_mean10year',
                                               'cumulative_mean15year'])]
features_to_drop.extend(cum_mean_to_drop)
raw_to_drop = ['total_pesticide_lbs', 'total_acres_treated']
features_to_drop.extend(raw_to_drop)
complete_dataset_final = complete_dataset_cleaned.drop(columns=features_to_drop)


print(f'Remaining: {[col for col in complete_dataset_final.columns if any(x in col for x in ['pesticide', 'acres', 'population'])]}')


Remaining: ['total_population', 'total_pesticide_lbs_per_100k', 'total_acres_treated_per_100k', 'total_pesticide_lbs_per_100k_lag1', 'total_pesticide_lbs_per_100k_lag2', 'total_acres_treated_per_100k_lag1', 'total_acres_treated_per_100k_lag2', 'total_pesticide_lbs_per_100k_cumulative_mean5year', 'total_pesticide_lbs_per_100k_cumulative_mean20year', 'total_acres_treated_per_100k_cumulative_mean5year', 'total_acres_treated_per_100k_cumulative_mean20year']


In [17]:
# Data prep for XGBoost

pesticide_features = ['total_population', 'total_pesticide_lbs_per_100k',
                      'total_acres_treated_per_100k', 'total_pesticide_lbs_per_100k_lag1',
                      'total_pesticide_lbs_per_100k_lag2', 'total_acres_treated_per_100k_lag1',
                      'total_acres_treated_per_100k_lag2',
                      'total_pesticide_lbs_per_100k_cumulative_mean5year',
                      'total_pesticide_lbs_per_100k_cumulative_mean20year',
                      'total_acres_treated_per_100k_cumulative_mean5year',
                      'total_acres_treated_per_100k_cumulative_mean20year']

confounders = ['median aqi', 'pct_under_18', 'pct_18_64', 'pct_65_plus', 'median_age',
               'pct_ai/an', 'pct_asian', 'pct_black', 'pct_latino', 'pct_multi_race',
               'pct_nh/pi', 'pct_white', 'poverty_allages_percent_est',
               'median_household_income_est']

all_features = pesticide_features + confounders

X = complete_dataset_final[all_features]
Y = complete_dataset_final['copd_hospitalization_rate']

print(f'X shape: {X.shape}')
print(f'Y shape: {Y.shape}')

train_df, test_df = train_test_split(
    complete_dataset_final,
    test_size=0.2,
    random_state=42
)


X_train = train_df[all_features].values
y_train = train_df['copd_hospitalization_rate'].values

X_test = test_df[all_features].values
y_test = test_df['copd_hospitalization_rate'].values

print(f'\nTarget variable (COPD rate) statistics:')
print(f' Train - Mean: {y_train.mean():.2f}, Std: {y_train.std():.2f}, Range: [{y_train.min():.2f}, {y_train.max():.2f}]')
print(f' Test  - Mean: {y_test.mean():.2f}, Std: {y_test.std():.2f}, Range: [{y_test.min():.2f}, {y_test.max():.2f}]')


X shape: (943, 25)
Y shape: (943,)

Target variable (COPD rate) statistics:
   Train - Mean: 16.22, Std: 8.42, Range: [1.62, 53.83]
   Test  - Mean: 16.77, Std: 10.16, Range: [2.57, 58.65]


In [18]:
# Baseline XGBoost Model Training

model = xgb.XGBRegressor(
    objective='reg:squarederror',
    learning_rate=0.05,
    max_depth=3,
    n_estimators=50,
    min_child_weight=10,
    subsample=0.7,
    colsample_bytree=0.7,
    reg_alpha=1.0,
    reg_lambda=1.0,
    random_state=42
)


model.fit(X_train,y_train, verbose=False)
y_train_prediction = model.predict(X_train)
y_test_prediction = model.predict(X_test)

rmse_training = np.sqrt(mean_squared_error(y_train,y_train_prediction))
mae_training = (mean_absolute_error(y_train,y_train_prediction))
r2_score_training = r2_score(y_train,y_train_prediction)

rmse_testing = np.sqrt(mean_squared_error(y_test,y_test_prediction))
mae_testing = mean_absolute_error(y_test,y_test_prediction)
r2_score_testing = r2_score(y_test,y_test_prediction)


r2_diff = r2_score_training - r2_score_testing
print(f'Train R² - Test R² = {r2_diff:.4f}')

if r2_diff > 0.2:
  print(f'\nOverfitting Detected (R² gap: {r2_diff:.4f})')
elif r2_diff > 0.1:
  print(f'Moderate Overfitting (R² gap: {r2_diff:.4f})')
else:
  print(f'Minimal overfitting (R² gap: {r2_diff:.4f})')


naive_rmse = np.sqrt(mean_squared_error(y_test, [y_train.mean()]*len(y_test)))
improvement = (naive_rmse - rmse_testing) / naive_rmse * 100

print(f'\nBaseline comparison:')
print(f'Naive (predict train mean): RMSE ={naive_rmse:.2f}')
print(f'XGBoost: RMSE = {rmse_testing:.2f}')
print(f'Improvement: {improvement:.1f}%')

print('\nTraining set performance:')
print(f' RMSE: {rmse_training:.2f}')
print(f' MAE:  {mae_training:.2f}')
print(f' R²: {r2_score_training:.4f}')

print('\nTest set performance:')
print(f' RMSE: {rmse_testing:.2f}')
print(f' MAE: {mae_testing:.2f}')
print(f' R²: {r2_score_testing:.4f}')

print(f'\nModel Performance Summary:')
if r2_score_testing < 0:
    print(f'Test R² is negative ({r2_score_testing:.4f})')
    print(f'Model performs worse than predicting the mean')
    print(f'Cause: Distribution shift between train/test')
elif r2_score_testing < 0.3:
    print(f'Test R² is low ({r2_score_testing:.4f})')
    print(f'Weak predictive power')
else:
    print(f'Test R² is {r2_score_testing:.4f}')
    print(f'Explains {r2_score_testing*100:.1f}% of variance')




Train R² - Test R² = 0.1375
Moderate Overfitting
   Test R² is GOOD (0.5878)
   Model explains 58.8% of variance

Baseline comparison:
Naive (predict train mean): RMSE =10.17
XGBoost: RMSE = 6.52
Improvement: 35.9%

Training set performance:
Root Mean Squared Error: 4.41
Mean Average Error:  3.18
R²: 0.7253

Test Set Performance:
Root Mean Squared Error: 6.52
Mean Average Error: 4.48
R²: 0.5878

Model Performance Summary:
Test R² is 0.5878
Explains 58.8% of variance


In [19]:
# Feature Importance

feat_importance = model.feature_importances_
df_importance = pd.DataFrame({'feature': all_features, 'importance': feat_importance}).sort_values('importance', ascending=False)

for idx, row in df_importance.head(15).iterrows():
    if row['feature'] in pesticide_features:
        category = '[PESTICIDE]'
    else:
        category = '[CONFOUNDER]'

    print(f'{row['feature']:50s} {category:13s} {row['importance']:.4f}')

pesticide_importance = df_importance[df_importance['feature'].isin(pesticide_features)]['importance'].sum()
confounder_importance = df_importance[df_importance['feature'].isin(confounders)]['importance'].sum()
total = pesticide_importance + confounder_importance


print(f'Pesticide features:  {pesticide_importance:.3f} ({pesticide_importance/total*100:.1f}%)')
print(f'Confounder features: {confounder_importance:.3f} ({confounder_importance/total*100:.1f}%)')

print('\nPesticide contribution to COPD prediction:')
if pesticide_importance/total > 0.30:
  print(f'Strong evidence: Pesticides account for {pesticide_importance/total*100:.1f}% of predictive power')
elif pesticide_importance/total > 0.15:
  print(f'Moderate evidence: Pesticides account for {pesticide_importance/total*100:.1f}% of predictive power')
elif pesticide_importance/total > 0.05:
  print(f'Weak evidence: Pesticides account for {pesticide_importance/total*100:.1f}% of predictive power')


total_acres_treated_per_100k_cumulative_mean20year [PESTICIDE]   0.1361
median_household_income_est                        [CONFOUNDER]  0.1097
total_pesticide_lbs_per_100k_cumulative_mean20year [PESTICIDE]   0.0999
pct_ai/an                                          [CONFOUNDER]  0.0785
total_acres_treated_per_100k_cumulative_mean5year  [PESTICIDE]   0.0520
poverty_allages_percent_est                        [CONFOUNDER]  0.0507
total_acres_treated_per_100k                       [PESTICIDE]   0.0411
pct_white                                          [CONFOUNDER]  0.0395
pct_asian                                          [CONFOUNDER]  0.0332
pct_under_18                                       [CONFOUNDER]  0.0319
median_age                                         [CONFOUNDER]  0.0303
pct_65_plus                                        [CONFOUNDER]  0.0291
total_pesticide_lbs_per_100k_lag1                  [PESTICIDE]   0.0284
pct_multi_race                                     [CONFOUNDER] 